# Langchain use of Chains

In [ ]:
import os
from google import genai
from langchain_google_genai import ChatGoogleGenerativeAI

# 1. Setup your key
os.environ["GEMINI_API_KEY"] = "XXX"
# 2. Initialize the model 
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")



## Single Chain

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 2. Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "Answer in one sentence: {question}"),
    ]
)

# 3. Chain: prompt → llm → parser
chain = prompt | llm | StrOutputParser()

# 4. Call the chain
result = chain.invoke({"question": "What is the largest ecommerce online retail store?"})
print(result)


Amazon is the largest ecommerce online retail store.


## Simple Sequential Chain

### Single Input and Single Output Pipeline

In [15]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

company_template = """\
    What is most notable name of \
    a company that {does_what}?
    """

description_template = """\
    Write a description for the following {company_name} and \
    and what is the crown jewel among it's subsidaries.
    """

# Chain 1: Company Name
first_prompt = ChatPromptTemplate.from_template(company_template)

first_chain = first_prompt | llm | StrOutputParser()

# Chain 2: Company -> Description
second_prompt = ChatPromptTemplate.from_template(description_template)

second_chain = second_prompt | llm | StrOutputParser()

# Overall Chain
overall_chain = {"company_name":first_chain} | second_chain


description = overall_chain.invoke({"does_what":"Sells things online"})
print(description)

Amazon arguably reigns supreme in the online marketplace, a name synonymous with e-commerce itself. Its global reach spans countless countries, serving a massive customer base and holding a dominant market share. The brand is instantly recognizable worldwide, offering an unparalleled variety of products, from groceries to gadgets. More than just a store, Amazon has revolutionized online shopping with innovations like one-click ordering, Prime membership, and a thriving third-party marketplace. While competitors like eBay, Walmart, and Alibaba are significant players, Amazon's overall influence and pioneering spirit solidify its position as the most notable name in online retail.

And arguably, the crown jewel among its subsidiaries is **Amazon Web Services (AWS)**. While Amazon's retail arm is its most visible face, AWS is a powerhouse of cloud computing, providing infrastructure and services to countless businesses and driving significant revenue and profit for the company.


## Sequential Chain
### Multi-input and Multi-output Pipeline

In [18]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# Translate -> English Translated Review
translate_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to English:\n\n{review}"
)

translate_chain = translate_prompt | llm | StrOutputParser()

# Summarize English Translated Review
summary_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence: \n\n{english_translated_review}"
)

summary_chain = summary_prompt | llm | StrOutputParser()

# Review -> Check Language
language_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{review}"
)

language_chain = language_prompt | llm | StrOutputParser()


from langchain_core.runnables import RunnablePassthrough

overall_chain = (
    {
        "language": language_chain,
        "english_translated_review": translate_chain,
        "review": RunnablePassthrough()  # Pass through original review
    }
    | RunnablePassthrough.assign(summary=summary_chain)
)

# Usage
review = """
デューク大学は、アメリカ・ノースカロライナ州ダーラムに本部を置く、世界トップクラスの私立研究大学です。1838年創立で、広大なキャンパスと、研究・医療・教育の分野で高い評価を受けており、\
特に「ブルーデビル」のマスコットとネイビーブルーのスクールカラーで知られ、世界大学ランキングでも常に上位に位置する名門校です。 \
主な特徴 \
所在地: アメリカ合衆国ノースカロライナ州ダーラム。 \
歴史: 1838年にメソジスト教会の附属学校として設立され、1924年にタバコ・発電事業で成功したジェームズ・ブキャナン・デューク氏の寄付により、現在の名前に改称されました。\
研究と教育: 優れた研究実績を持ち、医学、法学、ビジネス（フュークア・スクール）、海洋学など多岐にわたる分野で高い評価を得ています。\
キャンパス: ダーラム市郊外に広大な敷地（約8,600エーカー）を持ち、ウェスト、イースト、セントラルキャンパス、医療センターなどで構成され、無料バスで結ばれています。\
大学ランキング: 世界的に有名な大学であり、数々の世界大学ランキングで上位を占めています。\
研究連携: ローリーにあるノースカロライナ州立大学やチャペルヒルにあるノースカロライナ大学と共に「リサーチ・トライアングル」を形成しています。 \
デューク大学は、質の高い教育と研究、そしてサービスへの強いコミットメントを持ち、世界中から優秀な学生が集まる、歴史と未来を兼ね備えた教育機関です。"""
result = overall_chain.invoke({"review": review})
result

{'language': 'The language of the review is **Japanese**.\n\nYou can tell by the combination of:\n*   **Kanji** (e.g., 大学, 世界, 研究, 歴史)\n*   **Hiragana** (e.g., は, を, です, の)\n*   **Katakana** (e.g., デューク, アメリカ, ノースカロライナ, ブルーデビル)',
 'english_translated_review': '**Duke University** is a world-class private research university headquartered in Durham, North Carolina, USA. Founded in 1838, it boasts an expansive campus and holds a high reputation in the fields of research, medicine, and education. It is particularly known for its "Blue Devil" mascot and navy blue school colors, and is a prestigious institution consistently ranking among the top universities worldwide.\n\n**Key Features:**\n\n*   **Location:** Durham, North Carolina, United States.\n*   **History:** Established in 1838 as an affiliated school of the Methodist Church, it was renamed to its current name in 1924 thanks to a significant donation from James Buchanan Duke, who had found success in the tobacco and power generation

Step 1: Parallel Execution (Dictionary)
The dictionary {} creates a RunnableParallel that executes three operations simultaneously on the input:

"language": language_chain - Detects what language the review is written in

"english_translated_review": translate_chain - Translates the review to English

"review": RunnablePassthrough() - Preserves the original review text unchanged


Step 2: Enrich with Summary
The pipe operator | passes the dictionary to RunnablePassthrough.assign(summary=summary_chain).

The .assign() method adds a new key while preserving all existing keys:

Takes the english_translated_review from the dictionary

Runs it through summary_chain to generate a summary

Adds the result as a new "summary" key

Keeps all original keys intact


## Router + MultiPromptChain → routing to subchains
### Concept
A “router chain” uses an LLM to decide which subchain to call based on the input.
​
In the notebook:

You define several subject-specific prompts (physics, math, history, CS).

A router LLM produces JSON with "destination" (which prompt) and "next_inputs" (possibly edited question).

MultiPromptChain calls the selected destination chain, or a default chain if nothing matches.
​

In [30]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# Define prompt templates
physics_template = """ You are a very smart physics professor. \n
You are great at answering questions about physics and easy to understand 
in a concise manner.

Here is a question:
{input}
"""
math_template = """You are a very good mathematician.
You are great at answering math questions.
You are so good because you are able to break down
hard problems into their component parts,
answer the component parts, and then put them together
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian.
You have excellent knowledge of historical periods...
Here is a question:
{input}"""

computerscience_template = """You are a successful computer scientist.
You are great at answering coding questions...
Here is a question:
{input}"""

# Create prompt chains
physics_chain = ChatPromptTemplate.from_template(physics_template) | llm | StrOutputParser()
math_chain = ChatPromptTemplate.from_template(math_template) | llm | StrOutputParser()
history_chain = ChatPromptTemplate.from_template(history_template) | llm | StrOutputParser()
cs_chain = ChatPromptTemplate.from_template(computerscience_template) | llm | StrOutputParser()

# Create a classification prompt
classifier_template = """Given the user question below, classify it as either being about \
'physics', 'math', 'history', or 'computer science'.

Return ONLY ONE WORD in lowercase: physics, math, history, or computer science.

Question: {input}
Classification:"""

classifier_prompt = ChatPromptTemplate.from_template(classifier_template)
classifier_chain = classifier_prompt | llm | StrOutputParser()

# Create the router using RunnableBranch
router_chain = (
    {
        "topic": classifier_chain,
        "input": lambda x: x["input"]
    }
    | RunnableBranch(
        (lambda x: "physics" in x["topic"].lower(), physics_chain),
        (lambda x: "math" in x["topic"].lower(), math_chain),
        (lambda x: "history" in x["topic"].lower(), history_chain),
        (lambda x: "computer" in x["topic"].lower(), cs_chain),
        physics_chain  # Default fallback
    )
)

# Usage
result = router_chain.invoke({"input": "What is Newton's second law?"})
print(result)

result = router_chain.invoke({"input": "Who is Eugene Wong from Duke University?"})
print(result)

Ah, Newton's Second Law! A cornerstone of classical mechanics.

In short: **The net force acting on an object is equal to the product of its mass and its acceleration.**

Mathematically, we express this as:

$\mathbf{F}_{\text{net}} = m\mathbf{a}$

Where:
*   $\mathbf{F}_{\text{net}}$ is the **net force** (the vector sum of all forces) acting on the object, measured in Newtons (N).
*   $m$ is the **mass** of the object, measured in kilograms (kg).
*   $\mathbf{a}$ is the **acceleration** of the object, measured in meters per second squared (m/s²).

Essentially, it tells us that a force causes an object to accelerate, and the acceleration is directly proportional to the force and inversely proportional to the object's mass. The acceleration always points in the same direction as the net force.
Ah, Eugene Wong! A name that immediately brings to mind some foundational work in the world of stochastic processes and control theory.

Eugene Wong is a highly distinguished figure, particularly 

### RunnableBranch evaluates conditions sequentially and executes the first matching runnable.
​
#### How RunnableBranch works:
​
Takes a list of (condition_function, runnable) tuples

Evaluates each condition function with the input dictionary

Runs the runnable from the first condition that returns True

If no conditions match, runs the default (last parameter)